In [1]:
import tensorflow as tf

#### 1. 使用滑动平均。
* [class tf.train.ExponentialMovingAverage](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/train/ExponentialMovingAverage#__init__)
* __init__(decay, num_updates=None, zero_debias=False, name='ExponentialMovingAverage')
* $${shadow\_variable} = decay * shadow\_variable + (1 - decay) * variable$$
* when num_updates aviliable:
$$decay=min\lbrace {decay}, \frac{1+num\_updates}{10+num\_updates}\rbrace$$

In [2]:
v = tf.Variable(0, dtype=tf.float32, name="v")
# 在没有声明滑动平均模型时只有变量‘v’
for variables in tf.global_variables(): 
    print variables.name # 'v:0'
    
# ExponentialMovingAverage
ema = tf.train.ExponentialMovingAverage(0.99) # class
maintain_averages_op = ema.apply(tf.global_variables()) # operation
# 声明滑动平均模型后，会自动生成影子变量：'v/ExponentialMovingAverage:0'
print "\nExponentialMovingAverage:"
for variables in tf.global_variables(): 
    print variables.name

v:0

ExponentialMovingAverage:
v:0
v/ExponentialMovingAverage:0


#### 2. 保存滑动平均模型。

In [3]:
# Saver class
saver = tf.train.Saver() # Saves and restores variables.
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    sess.run(tf.assign(v, 10)) # v=10
    # 更新滑动平均，decay=min(0.99,(1+step)/(10+step)=0.1)=0.1
    # average(v) = decay*shadow_variable+(1-decay)*variable = 0.1*0+0.9*10=9
    sess.run(maintain_averages_op) # 
    # 保存的时候会将v:0  v/ExponentialMovingAverage:0这两个变量都存下来。
    saver.save(sess, "saved_model/model2.ckpt")
    # print `v` & `v的滑动平均`
    print sess.run([v, ema.average(v)]) # [10.0, 0.099999905] 0.99?

[10.0, 0.099999905]


#### 3. 加载滑动平均模型。

In [4]:
v = tf.Variable(0, dtype=tf.float32, name="v")

# 通过变量重命名将原来变量v的滑动平均值直接赋值给v。
saver = tf.train.Saver({"v/ExponentialMovingAverage": v})
with tf.Session() as sess:
    saver.restore(sess, "saved_model/model2.ckpt")
    print sess.run(v)

INFO:tensorflow:Restoring parameters from saved_model/model2.ckpt
0.099999905
